# 머릿말

각각의 user / scene별로 별도의 csv파일로 저장하는걸 목적

user7-scene1 같은 경우는 둘로 나눠지기 때문에, 파일 두 개를 만든다.
user-scene-index 순서

In [1]:
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
from math import pi
import pandas as pd

In [2]:
ds_split = {
    11: [(2000, 11000), (12000, 21000)],
    12: [(2000, 5000 ), (6000 , 10000)],
    13: [(2000, 10000), (11000, 18000)],
    21: [(2000, 11000), (12000, 20000)],
    22: [(2000, 6000 ), (7000 , 11000)],
    23: [(2000, 11000), (12000, 20000)],
    31: [(2000, 11000), (12000, 20000)],
    32: [(2000, 5000 ), (6000 , 11000)],
    33: [(2000, 10000), (11000, 19000)],
    41: [(2000, 10000), (11000, 19000)],
    42: [(2000, 6000 ), (7000 , 10000)],
    43: [(2000, 10000), (11000, 19000)],
    51: [(2000, 11000), (12000, 20000)],
    52: [(2000, 6000 ), (7000 , 10000)],
    53: [(2000, 10000), (11000, 19000)],
    61: [(2000, 9000 ), (10000, 17000)],
    62: [(2000, 11000), (12000, 20000)],
    63: [(2000, 10000), (11000, 19000)],
    71: [(2000, 7000 ), (9000 , 19000)],
    72: [(2000, 9000 ), (10000, 17000)],
    73: [(2000, 10000), (11000, 19000)]
}

### 각각의 User/Scene별로 데이터가 몇개씩 있는지

생각외로 고르게 분포되어있음

In [3]:
users_train, users_test = defaultdict(int), defaultdict(int)
scenes_train, scenes_test = defaultdict(int), defaultdict(int)

In [4]:
for key in ds_split.keys():
    user = key // 10
    scene = key % 10
    
    s, t = ds_split[key][0]
    users_train[user] += t - s
    scenes_train[scene] += t - s
    
    s, t = ds_split[key][1]
    users_test[user] += t - s
    scenes_test[scene] += t - s

In [5]:
users_train, users_test

(defaultdict(int,
             {1: 20000,
              2: 22000,
              3: 20000,
              4: 20000,
              5: 21000,
              6: 24000,
              7: 20000}),
 defaultdict(int,
             {1: 20000,
              2: 20000,
              3: 21000,
              4: 19000,
              5: 19000,
              6: 23000,
              7: 25000}))

In [6]:
scenes_train, scenes_test

(defaultdict(int, {1: 56000, 2: 34000, 3: 57000}),
 defaultdict(int, {1: 58000, 2: 34000, 3: 55000}))

## 데이터 생성

In [12]:
COLUMNS = ['timestamp', 'acceleration_x', 'acceleration_y', 'acceleration_z', 
           'input_orientation_yaw', 'input_orientation_pitch', 'input_orientation_roll',
           'input_orientation_x', 'input_orientation_y', 'input_orientation_z', 'input_orientation_w',
           'magnetic_x', 'magnetic_y', 'magnetic_z',
           'angular_vec_x', 'angular_vec_y', 'angular_vec_z']

In [13]:
mkdir data/1220

mkdir: cannot create directory ‘data/1220’: File exists


In [14]:
all_len, all_yaw, all_pitch, all_roll, all_x, all_y, all_z = 0, 0, 0, 0, 0, 0, 0

for key in ds_split.keys():
    user = key // 10
    scene = key % 10
    csv_fname = f'data/org/motion_data_user{user}_scene{scene}.csv'
    csv = pd.read_csv(csv_fname)
    
    csv = csv[COLUMNS]
    
    # timestamp flick ==> sec
    csv.timestamp = (csv.timestamp - csv.timestamp[2000]) / 705600000
    
    # yaw, pitch, roll radian ==> degree
    csv.input_orientation_yaw *= 180 / pi
    csv.input_orientation_pitch *= 180 / pi
    csv.input_orientation_roll *= 180 / pi
    csv.angular_vec_x *= 180 / pi
    csv.angular_vec_y *= 180 / pi
    csv.angular_vec_z *= 180 / pi
    csv.input_orientation_x *= 180 / pi
    csv.input_orientation_y *= 180 / pi
    csv.input_orientation_z *= 180 / pi
    csv.input_orientation_w *= 180 / pi
    
    T = csv.timestamp
    dT = T[:-1].to_numpy() - T[1:].to_numpy()
    dT = np.insert(dT, 0, dT[0])
    
    for i, (S, T) in enumerate(ds_split[key]):
        dsv_fname = f'data/1220/user{user}_scene{scene}_{i}.csv'
        print(dsv_fname)
        dsv = csv.iloc[S:T]
        dsv.to_csv(dsv_fname, index=False)

AttributeError: 'DataFrame' object has no attribute 'timestamp'